In [113]:
import csv
import numpy as np
import re
import gc
from colorama import Fore, Back, Style
from collections import defaultdict

In [114]:
solution_file = open("./processed_data/FinalSolutions-multilabel-argouml.txt")
solutions = []
for line in solution_file:
    line = line.lower()
#     line = line.replace("nan", "")
    line = line.replace("\n", "")
    solutions.append(str(line))

In [115]:
solution_str = []
objectives = []
for dd in solutions:
    num_str = dd[-16:]
    int_numbers = [int(nn) for nn in (re.findall('\d+', num_str ))]
    strs = dd[:-16]
    if len(int_numbers)==6:
        solution_str.append(strs)
        objectives.append(np.array(int_numbers))
    else:
        continue
del solutions
gc.collect()

0

In [116]:
len(solution_str), len(objectives)

(5450, 5450)

In [117]:
# Refactoring operations
tmp = ["Move Method", "Move Field", "Extract Class", "Push Down Field", "Push Down Method", "Extract Sub Class",\
       "Pull Up Field", "Pull Up Method", "Extract Super Class", "Encapsulate Field", "Increase Field Security",\
       "Decrease Field Security", "Increase Method Security", "Decrease Method Security"]

refactoring_operations = [refop.lower().replace(" ", "") for refop in tmp]

In [118]:
# number of important features
num_important = 50

important_features = dict()
with open('./feature_ranking/important_features-ant.csv') as csvfile: 
    csv_reader = csv.reader(csvfile)
    for line in csv_reader: 
        important_features[line[0]] = line[1:num_important + 1]


In [119]:
important_features

{'Effectiveness': ['movefield',
  'movemethod',
  'orgapachetoolsanttaskdefstypedef',
  'orgapachetoolsanttaskdefsmanifestexception',
  'orgapachetoolsanttypesfilelist',
  'orgapachetoolsanttypesfilterchain',
  'increasefieldsecurity',
  'encapsulatefield',
  'orgapachetoolsanttypesdirset',
  'orgapachetoolsanttypesselectorsmodifiedselectormodifiedselector',
  'orgapachetoolsanttypesselectorsbaseselectorcontainer',
  'orgapachetoolsanttarget',
  'orgapachetoolsanthelperantxmlcontext',
  'orgapachetoolsanttypesfilterset',
  'orgapachetoolsantutilstringtokenizer',
  'orgapachetoolsanttypesresourcesselectorsdate',
  'orgapachetoolsanttypesregularexpression',
  'orgapachetoolsanttypesselectorsmodifiedselectorhashvaluealgorithm',
  'orgapachetoolsanttypesselectorsmodifiedselectorpropertiesfilecache',
  'orgapachetoolsanttypesfileset',
  'orgapachetoolsanttypesselectorsmodifiedselectordigestalgorithm',
  'orgapachetoolsantutilfiletokenizer',
  'orgapachetoolsantprojecthelper',
  'orgapacheto

In [120]:
# solution number 
smallidx = 101
for idx, sol in enumerate(solution_str): 
    if len(sol.split(" ")) < len(solution_str[smallidx].split(" ")): 
        smallidx = idx

solution_num = smallidx
solution = [solution_str[solution_num], objectives[solution_num]]
solution[0]

'extractsuperclass org.argouml.ocl.argoany extractclass org.argouml.uml.diagram.collaboration.collabdiagramgraphmodel decreasefieldsecurity org.argouml.taskmgmt.progressevent extractclass org.argouml.uml.cognitive.projectmembertodolist extractsubclass org.argouml.util.osdep.osxadapter extractsubclass org.argouml.uml.reveng.ui.problemsdialog increasemethodsecurity org.argouml.ui.targetmanager.targetmanager extractclass org.argouml.uml.diagram.state.statediagramgraphmodel extractsuperclass class_7 extractclass org.argouml.ui.explorer.dndexplorertree decreasefieldsecurity org.argouml.gefext.argofigink extractclass org.argouml.util.osdep.osutil extractsubclass org.argouml.language.ui.languagecombobox decreasemethodsecurity org.argouml.uml.diagram.state.ui.figtransition extractsubclass org.argouml.ui.finddialog extractclass org.argouml.uml.diagram.state.ui.figjunctionstate extractsuperclass org.argouml.language.ui.languagecombobox extractclass org.argouml.notation.providers.java.association

In [121]:
improved_objectives = np.where(solution[1] == 1)[0]
improved_features = [list(important_features.keys())[i] for i in improved_objectives]
print(f"Solution {solution_num} improved the following QMOODs:")
improved_features

Solution 3489 improved the following QMOODs:


['Functionality', 'Understandability']

In [122]:
# stripping the .'s 
explanation = defaultdict(list)

print("Summary: \n")
for qmood in improved_features:
    print(f"The refactoring solution improved {qmood}.")
    flag = True
    for striped_oploc in important_features[qmood]:
        if striped_oploc in solution[0].replace(".", ""):
            flag = False
            if striped_oploc in refactoring_operations: 
                print(f"\tBecause it had {striped_oploc} as a refactoring operation.")
            else: 
                print(f"\tBecause it had operated on the location: {striped_oploc}.")
            explanation[striped_oploc].append(qmood)
    if flag:
        print("\tBut no correlation could be made.")

explanation = dict(explanation)

Summary: 

The refactoring solution improved Functionality.
	Because it had extractsuperclass as a refactoring operation.
	Because it had extractsubclass as a refactoring operation.
	Because it had decreasemethodsecurity as a refactoring operation.
The refactoring solution improved Understandability.
	Because it had extractsuperclass as a refactoring operation.
	Because it had extractclass as a refactoring operation.
	Because it had extractsubclass as a refactoring operation.


In [123]:
explanation

{'extractsuperclass': ['Functionality', 'Understandability'],
 'extractsubclass': ['Functionality', 'Understandability'],
 'decreasemethodsecurity': ['Functionality'],
 'extractclass': ['Understandability']}

In [124]:
# print highlited solution
# settings colors
tmp = [Fore.BLUE, Fore.GREEN, Fore.MAGENTA, Fore.RED, Fore.LIGHTRED_EX, Fore.LIGHTBLACK_EX]
colors = dict(zip(improved_features, tmp))

for oploc in solution[0].split():
    striped_oploc = oploc.replace(".", "")
    # if oploc is important print with colors
    if striped_oploc in explanation.keys():
        # the qmood improved by oploc
        tmpl = explanation[striped_oploc]
        # important for only 1 qmood ==> 1 color
        if len(tmpl) == 1: 
            color = colors[tmpl[0]]
            print(color + oploc, end=" ")
        
        # important for many qmoods ==> many colors
        elif len(tmpl) != 0: 
            # dividing into pieces
            l = len(oploc) // len(tmpl)
            iters = [iter(oploc)] * l
            pieces = ["".join(piece) for piece in zip(*iters)]
            if len(oploc) % l != 0:
                pieces[-1] = pieces[-1] + oploc[-(len(oploc) % l):]
            for piece, color in zip(pieces, [colors[qmood] for qmood in tmpl]):
                print(color + piece, end="")
            
            print(end=" ")
            
    else:
         print(Fore.BLACK + oploc, end=" ")

extractsuperclass org.argouml.ocl.argoany extractclass org.argouml.uml.diagram.collaboration.collabdiagramgraphmodel decreasefieldsecurity org.argouml.taskmgmt.progressevent extractclass org.argouml.uml.cognitive.projectmembertodolist extractsubclass org.argouml.util.osdep.osxadapter extractsubclass org.argouml.uml.reveng.ui.problemsdialog increasemethodsecurity org.argouml.ui.targetmanager.targetmanager extractclass org.argouml.uml.diagram.state.statediagramgraphmodel extractsuperclass class_7 extractclass org.argouml.ui.explorer.dndexplorertree decreasefieldsecurity org.argouml.gefext.argofigink extractclass org.argouml.util.osdep.osutil extractsubclass org.argouml.language.ui.languagecombobox decreasemethodsecurity org.argouml.uml.diagram.state.ui.figtransition extractsubclass org.argouml.ui.finddialog extractclass org.argouml.uml.diagram.state.ui.figjunctionstate extractsuperclass org.argouml.language.ui.languagecombobox extractclass org.argouml.notation.providers.java.associationn